In [ ]:
import os
import rasterio
import numpy as np

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import config

import rsutils.utils as utils
import resample_worldcereal_cropmasks

In [ ]:
aez_s2_grid_folderpath = os.path.join(config.FOLDERPATH_OUTPUTS, 's2_grid_level/worldcereal/')
s2_grid_level_folderpath = os.path.join(config.FOLDERPATH_OUTPUTS, 's2_grid_level/aggregated_worldcereal/')
merged_product_folderpath = os.path.join(config.FOLDERPATH_OUTPUTS, 's2_grid_level/aggregated_worldcereal')
resampled_cropmasks_folderpath = os.path.join(config.FOLDERPATH_OUTPUTS, 's2_grid_level/resampled_cropmasks/')
output_folderpath = os.path.join(config.FOLDERPATH_OUTPUTS, 'merged_WC_cropmask/')

In [ ]:
WC_S2_GRID_RES = 4
WC_PRODUCTS_TO_MERGE = ['springcereals', 'wintercereals']
WC_MERGED_PRODUCT_NAME = 'cereals'
WC_OUTPRODUCTS = ['cereals', 'temporarycrops']
BINARY_MASK_SETTINGS = {
    'cereals': {
        'threshold': 10,
        'how': 'gte',
    },
    'temporarycrops': {
        'threshold': 0,
        'how': 'gt',
    }
}

In [ ]:
# 49 m 12.2 s
merged_cropmask_catalogue_df = \
resample_worldcereal_cropmasks.resample_worldcereal_cropmasks(
    roi_geom_filepath = config.FILEPATH_ROI_SHAPE,
    worldcereal_folderpath = config.FOLDERPATH_WORLDCEREAL,
    worldcereal_aez_filepath = config.FILEPATH_WORLDCEREAL_AEZ,
    ref_tif_filepath = config.FILEPATH_REFERENCE_GEOTIFF,
    aez_s2_grid_folderpath = aez_s2_grid_folderpath,
    s2_grid_level_folderpath = s2_grid_level_folderpath,
    merged_product_folderpath = merged_product_folderpath,
    resampled_cropmasks_folderpath = resampled_cropmasks_folderpath,
    output_folderpath = output_folderpath,
    products_to_merge = WC_PRODUCTS_TO_MERGE,
    merged_product_name = WC_MERGED_PRODUCT_NAME,
    out_products = WC_OUTPRODUCTS,
    s2_grid_res = WC_S2_GRID_RES,
    overwrite = False,
)

In [ ]:
merged_cropmask_catalogue_df

In [ ]:
tif_filepaths = dict(zip(
    merged_cropmask_catalogue_df['product'],
    merged_cropmask_catalogue_df['tif_filepath'],
))

tif_filepaths

In [ ]:
os.makedirs(config.FOLDERPATH_MASKS, exist_ok=True)

In [ ]:
def create_binary_mask(
    tif_filepath:str,
    out_mask_filepath:str,
    min_threshold,
    how='gte',
):
    with rasterio.open(tif_filepath) as src:
        ndarray = src.read()
        meta = src.meta.copy()
    
    out_ndarray = np.zeros(shape=ndarray.shape)
    if how == 'gte':
        out_ndarray[ndarray >= min_threshold] = 1
    elif how == 'gt':
        out_ndarray[ndarray > min_threshold] = 1
    else:
        raise ValueError(f"how must be either 'gte' or 'gt'.")

    _folderpath = os.path.split(out_mask_filepath)[0]
    os.makedirs(_folderpath, exist_ok=True)

    meta['dtype'] = rasterio.uint8
    meta['nodata'] = 0

    with rasterio.open(out_mask_filepath, 'w', **meta) as dst:
        dst.write(out_ndarray)

In [ ]:
for key, binmask_config in BINARY_MASK_SETTINGS.items():
    tif_filepath = tif_filepaths[key]
    out_mask_filepath = utils.modify_filepath(
        filepath=tif_filepath,
        new_folderpath=config.FOLDERPATH_MASKS,
    )
    create_binary_mask(
        tif_filepath=tif_filepath, 
        out_mask_filepath=  out_mask_filepath,
        min_threshold=binmask_config['threshold'],
        how=binmask_config['how'],
    )